<a href="https://colab.research.google.com/github/DBCooper200/project_dl/blob/main/resnet_modified_code.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [8]:
# Let's make sure the kaggle.json file is present.
!ls -lha kaggle.json

-rw-r--r-- 1 root root 67 May 21 17:06 kaggle.json


In [9]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [10]:
# Next, install the Kaggle API client.
!pip install -q kaggle

In [11]:
# The Kaggle API client expects this file to be in ~/.kaggle,
# so move it there.
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/

# This permissions change avoids a warning on Kaggle tool startup.
!chmod 600 ~/.kaggle/kaggle.json

In [12]:
!kaggle datasets download -d himanshuagarwal1998/glaucomadataset

 99% 2.52G/2.54G [00:27<00:00, 122MB/s] 
100% 2.54G/2.54G [00:28<00:00, 97.4MB/s]


In [13]:
!unzip -q /content/glaucomadataset.zip

In [14]:
!mkdir 'Data_set'

In [15]:
import shutil, sys
shutil.move("/content/Non Glaucoma", "/content/Data_set")

'/content/Data_set/Non Glaucoma'

In [16]:
shutil.move("/content/Glaucoma", "/content/Data_set")

'/content/Data_set/Glaucoma'

In [17]:
'''FAST_RUN = False
IMAGE_WIDTH=128
IMAGE_HEIGHT=128
IMAGE_SIZE=(IMAGE_WIDTH, IMAGE_HEIGHT)
IMAGE_CHANNELS=3
'''

'FAST_RUN = False\nIMAGE_WIDTH=128\nIMAGE_HEIGHT=128\nIMAGE_SIZE=(IMAGE_WIDTH, IMAGE_HEIGHT)\nIMAGE_CHANNELS=3\n'

In [18]:
import tensorflow as tf
import keras
from keras.models import Sequential
from keras.layers import Conv2D
from keras.layers import MaxPooling2D
from keras.layers import Flatten
from keras.layers import Dense
from tensorflow.keras.preprocessing import image
import os
import cv2
import numpy as np
from tqdm import tqdm
import warnings
warnings.filterwarnings('ignore')
import warnings
warnings.filterwarnings('ignore')

In [19]:
path = ['/content/Data_set/Non Glaucoma', '/content/Data_set/Glaucoma']
images = []
labels = []
for n,i in enumerate(path):
    for j in tqdm(os.listdir(i)):
        img_path = os.path.join(i,j)
        img = cv2.imread(img_path)
        img = cv2.resize(img, (64,64))
        images.append(img)
        labels.append(n)
images = np.array(images)/255
labels = np.array(labels)

100%|██████████| 511/511 [00:09<00:00, 51.30it/s]


In [20]:
for n in enumerate(path):
  print(n)

#the labels assigned are 
# 0 for non glaucomic images
# 1 for glaucomic images

(0, '/content/Data_set/Non Glaucoma')
(1, '/content/Data_set/Glaucoma')


In [21]:
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split

images, labels = shuffle(images, labels, random_state=32)
x_train, x_valid, y_train, y_valid = train_test_split(images, labels, test_size=0.15, random_state=44)
x_train, x_test, y_train, y_test = train_test_split(x_train, y_train, test_size=0.15, random_state=40)

In [22]:

from keras.preprocessing.image import ImageDataGenerator
datagen = ImageDataGenerator(zoom_range=0.2, horizontal_flip=True, vertical_flip=True,
                            fill_mode='constant', cval=0.)
train_gen = datagen.flow(x_train, y_train, batch_size=32)
#Using TensorFlow backend.

#for test set
test_gen = datagen.flow(x_test, y_test, batch_size = 32)

In [23]:
HEIGHT = 64
WIDTH = 64
BATCH_SIZE = 32

'\nclass_list = ["class_1", "class_2"]\nFC_LAYERS = [1024, 512, 256]\ndropout = 0.5\nNUM_EPOCHS = 10\nBATCH_SIZE = 32\n\ndef build_model(base_model, dropout, fc_layers, num_classes):\n    for layer in base_model.layers:\n        layer.trainable = False\n\n    x = base_model.output\n    x = Flatten()(x)\n    for fc in fc_layers:\n        print(fc)\n        x = Dense(fc, activation=\'relu\')(x)\n        x = Dropout(dropout)(x)\n    preditions = Dense(num_classes, activation=\'softmax\')(x)\n    finetune_model = Model(inputs = base_model.input, outputs = preditions)\n    return finetune_model\n\nbase_model_1 = ResNet50(weights = None,\n                       include_top = False,\n                       input_shape = (HEIGHT, WIDTH, 3))\n\ntrain_datagen = ImageDataGenerator(preprocessing_function = preprocess_input,\n                                   rotation_range = 90,\n                                   horizontal_flip = True,\n                                   vertical_flip = True,\n

In [24]:
import tensorflow as tf
from tensorflow.keras import Sequential, layers, Model
from tensorflow.keras.applications.resnet50 import ResNet50
'''
from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.applications.xception import Xception
from tensorflow.keras.applications.resnet import ResNet101
from tensorflow.keras.applications.inception_v3 import InceptionV3
from tensorflow.keras.applications.mobilenet_v2 import MobileNetV2
from tensorflow.keras.applications.efficientnet import EfficientNetB7
'''
from sklearn.metrics import classification_report
import plotly.express as px
import pandas as pd

In [25]:
IMAGE_SIZE = (64,64)

In [26]:
resnet =ResNet50(weights= None, include_top=False, input_shape= (HEIGHT,WIDTH,3))
#vgg = VGG16(input_shape=IMAGE_SIZE+(3,), weights='imagenet', include_top=False)
#xception = Xception(input_shape=IMAGE_SIZE+(3,), weights='imagenet', include_top=False)
#resnet101 = ResNet101(input_shape=IMAGE_SIZE+(3,), weights='imagenet', include_top=False)
#inception = InceptionV3(input_shape=IMAGE_SIZE+(3,), weights='imagenet', include_top=False)
#mobilenet = MobileNetV2(input_shape=IMAGE_SIZE+(3,), weights='imagenet', include_top=False)
#efficientnet = EfficientNetB7(input_shape=IMAGE_SIZE+(3,), weights='imagenet', include_top=False)

In [44]:
from keras.optimizers import SGD, Adam
from keras.layers import Dropout,Dense, GlobalAveragePooling2D
x = x = layers.Flatten()(resnet.output)
predictions = Dense(1,activation= 'softmax')(x)
model = Model(inputs = resnet.input, outputs = predictions)
model.compile(optimizer= 'adam', loss='categorical_crossentropy', metrics=['accuracy'])
model.summary()

Model: "model_4"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 64, 64, 3)]  0           []                               
                                                                                                  
 conv1_pad (ZeroPadding2D)      (None, 70, 70, 3)    0           ['input_1[0][0]']                
                                                                                                  
 conv1_conv (Conv2D)            (None, 32, 32, 64)   9472        ['conv1_pad[0][0]']              
                                                                                                  
 conv1_bn (BatchNormalization)  (None, 32, 32, 64)   256         ['conv1_conv[0][0]']             
                                                                                            

'.fit is used when the entire training dataset can fit into the memory \nand no data augmentation is applied.\n\nfit_generator is used when we have a huge dataset to fit into our \nmemory or when data augmentation needs to be applied.'

In [46]:
model.fit_generator(train_gen, epochs=100, steps_per_epoch=1,verbose=1, validation_data=(x_valid,y_valid),callbacks=None)

Epoch 1/100
1/1 [==============================] - 3s 3s/step - loss: 0.0000e+00 - accuracy: 0.4062 - val_loss: 0.0000e+00 - val_accuracy: 0.5390
Epoch 2/100
1/1 [==============================] - 0s 260ms/step - loss: 0.0000e+00 - accuracy: 0.4062 - val_loss: 0.0000e+00 - val_accuracy: 0.5390
Epoch 3/100
1/1 [==============================] - 0s 272ms/step - loss: 0.0000e+00 - accuracy: 0.5625 - val_loss: 0.0000e+00 - val_accuracy: 0.5390
Epoch 4/100
1/1 [==============================] - 0s 265ms/step - loss: 0.0000e+00 - accuracy: 0.5938 - val_loss: 0.0000e+00 - val_accuracy: 0.5390
Epoch 5/100
1/1 [==============================] - 0s 268ms/step - loss: 0.0000e+00 - accuracy: 0.5625 - val_loss: 0.0000e+00 - val_accuracy: 0.5390
Epoch 6/100
1/1 [==============================] - 0s 188ms/step - loss: 0.0000e+00 - accuracy: 0.5625 - val_loss: 0.0000e+00 - val_accuracy: 0.5390
Epoch 7/100
1/1 [==============================] - 0s 186ms/step - loss: 0.0000e+00 - accuracy: 0.5625 - val_

In [48]:
model.save('path_to_my_model.h5')

# Recreate the exact same model purely from the file
new_model = keras.models.load_model('path_to_my_model.h5')

In [49]:
pred = new_model.evaluate(x_test, y_test)

Exception ignored in: <function _xla_gc_callback at 0x7fd4221429e0>
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/jax/_src/lib/__init__.py", line 97, in _xla_gc_callback
    def _xla_gc_callback(*args):
KeyboardInterrupt: 


5/5 [==============================] - 57s 127ms/step - loss: 0.0000e+00 - accuracy: 0.5496
